# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [28]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

import math
import numpy as np
from collections import Counter, defaultdict
import os
import random
import re
from collections import Counter, defaultdict
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Dropout, Bidirectional, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Flatten
from tensorflow.keras import backend as K

In [3]:
def load_training_words(path="words_250000_train.txt", min_len=1, max_len=29):
    words = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            w = line.strip().lower()
           
            if w.isalpha() and min_len <= len(w) <= max_len:
                words.append(w)
    return words

training_words = load_training_words()
# print(f"Loaded {len(training_words)} words from training dictionary.")

alphabets = "abcdefghijklmnopqrstuvwxyz"
letter_index = {c: i for i, c in enumerate(alphabets)}
index_letter = {i: c for c, i in letter_index.items()}
min_len, max_len = min(len(w) for w in training_words), max(len(w) for w in training_words)

#deal with the encrypted GUESS word
def clean_mask(raw_string):
#remove spaces between and outside the string and 
    string = raw_string.replace(" ","").lower()
    return string

def mask_to_char_indices(string, pad_to=max_len):
#input is the cleaned string containing masked letters
#string is converted into encoded numbers corresponding to their indices
#the masked letters are given "26" as the index
#The encoded array is also padded to a specific length, to feed equal lengths of all words to the ML model
    unknown_index = len(alphabets)  
    seq = []
    for ch in string:
        if ch == "_":
            seq.append(unknown_index)
        elif ch in letter_index:
            seq.append(letter_index[ch])
        else:
            seq.append(unknown_index)
    # pad on right with unknown
    while len(seq) < pad_to:
        seq.append(unknown_index)
    if len(seq) > pad_to:
        seq = seq[:pad_to]
    return np.array(seq, dtype=np.int32)


#find out the most frequent letters in the dictionary and rank them
dict_letter_freq = Counter()
for w in training_words:
    # count unique letter occurrence per word to avoid bias by repeated letters
    for c in set(w):
        dict_letter_freq[c] += 1
dict_letter_rank = [x for x, _ in dict_letter_freq.most_common()]

In [31]:
#Simulate the fake games to create features for training
def sample_game_state_from_word(word, reveal_prob=0.25, max_wrong=4):
    #Decide which positions are revealed
    reveal_mask = [random.random() < reveal_prob for _ in word]
    masked = [word[i] if reveal_mask[i] else "_" for i in range(len(word))]
    #Start guessed set includes the revealed letters
    guessed = set([c for c in masked if c != "_"])
    #Add some wrong guesses randomly from alphabet excluding letters in word
    non_word_letters = [c for c in alphabets if c not in set(word)]
    num_wrong = random.randint(0, max_wrong)
    if non_word_letters and num_wrong > 0:
        guessed.update(random.sample(non_word_letters, min(num_wrong, len(non_word_letters))))
    mask_str = "".join(masked)
    return mask_str, guessed

def create_training_examples(words, n_examples):
    #Add features and target for training 
    #choose the not guesses letters and the ones with highest frequency
    #the masked array  with padding is also passed
    N = n_examples
    X_seq = np.zeros((N, max_len), dtype=np.int32)
    X_guessed = np.zeros((N, 26), dtype=np.int32)
    X_len = np.zeros((N,), dtype=np.int32)  # optional: actual word length
    y = np.zeros((N, 26), dtype=np.int32)
    i = 0
    attempts = 0
    while i < N and attempts < N * 10:
        attempts += 1
        w = random.choice(words)
        mask_str, guessed = sample_game_state_from_word(w, reveal_prob=random.uniform(0.05, 0.5), max_wrong=3)
        #compute remaining letters in the word that are not revealed or guessed
        remaining_letters = [c for c in sorted(set(w)) if c not in guessed and c not in mask_str]
        if not remaining_letters:
            continue  # nothing to predict
        #choose label: highest global-frequency among remaining letters
        remaining_by_freq = sorted(remaining_letters, key=lambda c: dict_letter_freq.get(c, 0), reverse=True)
        chosen = remaining_by_freq[0]
        X_seq[i, :] = mask_to_char_indices(mask_str, pad_to=max_len)
        for g in guessed:
            if g in letter_index:
                X_guessed[i, letter_index[g]] = 1
        X_len[i] = len(w)
        y[i, letter_index[chosen]] = 1
        i += 1
    # trim in case of early stop
    if i < N:
        X_seq = X_seq[:i]
        X_guessed = X_guessed[:i]
        X_len = X_len[:i]
        y = y[:i]
    return X_seq, X_guessed, X_len, y

X_seq, X_guessed, X_len, y = create_training_examples(training_words, n_examples=100000)

In [40]:
#create the LSTM training model

# def build_model(max_seq_len=max_len, vocab_size=len(alphabets)+1, emb_dim=32, lstm_units=128):
#     """
#     Inputs:
#       - sequence of char indices (vocab_size includes unknown token at index len(ALPHABET))
#       - guessed mask (26-dim)
#       - optionally length scalar (not used extensively)
#     """
#     seq_in = Input(shape=(max_seq_len,), name="seq_in")
#     guessed_in = Input(shape=(26,), name="guessed_in")
#     # embedding: vocab_size = 26 letters + unknown '_' token
#     x = Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=max_seq_len, mask_zero=False)(seq_in)
#     x = LSTM(lstm_units, return_sequences=False)(x)
#     x = Dropout(0.2)(x)
#     # combine with guessed mask
#     combined = Concatenate()([x, guessed_in])
#     dense = Dense(128, activation="relu")(combined)
#     out = Dense(26, activation="softmax")(dense)
#     model = Model(inputs=[seq_in, guessed_in], outputs=out)
#     model.compile(optimizer=Adam(1e-3), loss="categorical_crossentropy", metrics=["accuracy"])
#     return model

def build_bilstm_with_conv_bigrams(max_seq_len=max_len, vocab_size=len(alphabets)+1, emb_dim=128,
                                  lstm_units=[192,128], conv_filters=64, conv_kernel_sizes=[2], dropout=0.2):
    # Inputs
    seq_in = Input(shape=(max_seq_len,), name="seq_in")
    guessed_in = Input(shape=(26,), name="guessed_in")
    len_in = Input(shape=(1,), name="len_in")  # optional

    # Character embedding
    emb = Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=max_seq_len)(seq_in)

    # Conv1D branch(s) to capture bigrams (kernel_size=2 for bigrams)
    conv_outs = []
    for k in conv_kernel_sizes:                    # you can try [2] or [2,3]
        c = Conv1D(filters=conv_filters, kernel_size=k, padding="valid", activation="relu")(emb)
        c = GlobalMaxPooling1D()(c)
        conv_outs.append(c)
    conv_feat = conv_outs[0] if len(conv_outs) == 1 else Concatenate()(conv_outs)

    # BiLSTM stack on top of embeddings (as before)
    x = emb
    for i, units in enumerate(lstm_units):
        return_seq = (i < len(lstm_units)-1)
        x = Bidirectional(LSTM(units, return_sequences=return_seq, dropout=dropout, recurrent_dropout=0.1))(x)
    x = Dropout(dropout)(x)

    # Concatenate LSTM global features + conv bigram features + guessed mask + length
    x = Concatenate()([x, conv_feat, guessed_in, len_in])
    x = Dense(128, activation="relu")(x)
    x = Dropout(dropout)(x)
    out = Dense(26, activation="softmax", name="out")(x)
    schedule = ExponentialDecay(initial_learning_rate=3e-4, decay_steps=2000, decay_rate=0.9, staircase=False)
    loss = CategoricalCrossentropy(label_smoothing=0.02) 
    model = Model(inputs=[seq_in, guessed_in, len_in], outputs=out)
    model.compile(optimizer=Adam(learning_rate=schedule, clipnorm=1), loss=loss, metrics=["accuracy"])
    return model

model = build_bilstm_with_conv_bigrams() #build_model()
model.summary()


#save model and load it next time
# (5) Train (or load) the model
pretrained_model = "devanshu_hangman.h5"
if os.path.exists(pretrained_model):
    print("Loading existing model:", pretrained_model)
    model = load_model(pretrained_model)
else:
    # train-validation split
    N = X_seq.shape[0]
    idx = np.arange(N)
    np.random.shuffle(idx)
    split = int(0.85 * N)
    train_idx, val_idx = idx[:split], idx[split:]
    train_inputs = [X_seq[train_idx], X_guessed[train_idx]]
    val_inputs = [X_seq[val_idx], X_guessed[val_idx]]
    train_y = y[train_idx]
    val_y = y[val_idx]

    # callbacks
    cb = [ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-5),
          ModelCheckpoint(pretrained_model, monitor="val_loss", save_best_only=True, verbose=1)]

    epochs = 40 #if FAST_TRAIN else 25
    batch_size = 1024 #if FAST_TRAIN else 1024

    print("Starting training: epochs", epochs, "batch_size", batch_size)
    
    if 'X_len' not in globals():
    # Example: compute lengths from X_seq where 0 means padding
        X_len = np.sum(X_seq != 0, axis=1)

    # reshape to (N,1) because model expects shape (None,1)
    X_len = X_len.reshape(-1, 1)

    # Train-validation split (you already had this)
    N = X_seq.shape[0]
    idx = np.arange(N)
    np.random.shuffle(idx)
    split = int(0.85 * N)
    train_idx, val_idx = idx[:split], idx[split:]

    # Now include the length arrays in training/validation inputs (third item)
    train_inputs = [X_seq[train_idx], X_guessed[train_idx], X_len[train_idx]]
    val_inputs   = [X_seq[val_idx],   X_guessed[val_idx],   X_len[val_idx]]

    train_y = y[train_idx]
    val_y   = y[val_idx]


    model.fit(train_inputs, train_y, validation_data=(val_inputs, val_y), epochs=epochs, batch_size=batch_size, callbacks=cb)
    # best model saved by ModelCheckpoint
    model = load_model(pretrained_model)
    print("Model trained and loaded from", pretrained_model)


Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ seq_in (InputLayer) │ (None, 29)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_16        │ (None, 29, 128)   │      3,456 │ seq_in[0][0]      │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_28    │ (None, 29, 384)   │    493,056 │ embedding_16[0][… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_29    │ (None, 256)       │    525,312 │ bidirectional_28… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 28, 64)    │     16,448 │ embedding_16[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_28          │ (None, 256)       │          0 │ bidirectional_29… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 64)        │          0 │ conv1d_3[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ guessed_in          │ (None, 26)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ len_in (InputLayer) │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_15      │ (None, 347)       │          0 │ dropout_28[0][0], │
│ (Concatenate)       │                   │            │ global_max_pooli… │
│                     │                   │            │ guessed_in[0][0], │
│                     │                   │            │ len_in[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 128)       │     44,544 │ concatenate_15[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_29          │ (None, 128)       │          0 │ dense_15[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out (Dense)         │ (None, 26)        │      3,354 │ dropout_29[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,086,170 (4.14 MB)

 Trainable params: 1,086,170 (4.14 MB)

 Non-trainable params: 0 (0.00 B)

Loading existing model: devanshu_hangman.h5


In [41]:
class LSTMGuesser:
    def __init__(self, model, words):
        self.model = model
        self.words = words
        self.reset_game()

    def reset_game(self):
        self.guessed = set()
        self.last_mask = None
        self.wrong_guesses = 0
        self.candidate_cache = None
        self.current_word_length = None

    def _update_state_from_mask(self, mask):
        # mask is API-style with spaces; normalize to no-spaces
        m = clean_mask(mask)
        # if new game starts (length changes or last_mask is None), reset
        if self.last_mask is None or len(m) != len(self.last_mask):
            self.reset_game()
            self.current_word_length = len(m)
        # infer guessed letters from revealed characters in mask
        revealed_letters = set([c for c in m if c != "_" and c in alphabets])
        # guessed includes any revealed + previously guessed
        self.guessed.update(revealed_letters)
        # update wrong_guesses estimate: if guessed set size larger than revealed distinct letters discovered assume wrong guesses exist
        # note: we only estimate, server is authoritative, but this is enough for tracking
        self.last_mask = m

    def _filter_candidates(self, mask):
        # fast pattern match: create regex
        m = mask
        regex_pattern = "^" + m.replace("_", ".") + "$"
        prog = re.compile(regex_pattern)
        # filter words of the same length
        L = len(m)
        matched = [w for w in self.words if len(w) == L and prog.match(w)]
        return matched

    def _model_predict_proba(self, mask):
        seq = mask_to_char_indices(clean_mask(mask), pad_to=max_len)[None, :]
        guessed_vec = np.zeros((1, 26), dtype=np.int32)
        for g in self.guessed:
            if g in letter_index:
                guessed_vec[0, letter_index[g]] = 1
        probs = self.model.predict([seq, guessed_vec], verbose=0)[0]  # 26-dim
        return probs

    def guess(self, mask):
        #update game state
        self._update_state_from_mask(mask)

        #If the model is not available, fallback to dictionary frequency policy
        if self.model is None:
            return self._fallback_guess(mask)

        #Get model probabilities
        probs = self._model_predict_proba(mask)

        #Mask out already guessed letters
        for g in self.guessed:
            if g in letter_index:
                probs[letter_index[g]] = 0.0

        #If candidates are many, blend with candidate letter frequency
        candidates = self._filter_candidates(clean_mask(mask))
        # compute frequency over candidates
        candidate_counts = Counter()
        for w in candidates:
            for c in set(w):
                if c not in self.guessed and c not in clean_mask(mask):
                    candidate_counts[c] += 1
        # normalize to prob-like distribution
        candidate_total = sum(candidate_counts.values())
        if candidate_total > 0:
            candidate_probs = np.zeros(26)
            for c, cnt in candidate_counts.items():
                candidate_probs[letter_index[c]] = cnt / candidate_total
            # combine: weighted sum, favor model but include candidate info
            alpha = 0.7
            blended = alpha * probs + (1 - alpha) * candidate_probs
        else:
            blended = probs

        # pick argmax of blended
        idx = int(np.argmax(blended))
        chosen = index_letter[idx]

        # safety: if chosen already guessed (numerical/rounding), fallback to top unused from global rank
        if chosen in self.guessed:
            for c in dict_letter_freq:
                if c not in self.guessed:
                    chosen = c
                    break

        # update guessed
        self.guessed.add(chosen)
        return chosen

    def _fallback_guess(self, mask):
        # Most common unused global letters
        for c in dict_letter_freq:
            if c not in self.guessed:
                self.guessed.add(c)
                return c
        # final fallback
        for c in alphabets:
            if c not in self.guessed:
                self.guessed.add(c)
                return c
        # if nothing left
        return random.choice(alphabets)

# instantiate the guesser (model variable defined earlier)
try:
    guesser = LSTMGuesser(model, training_words)
    print("LSTMGuesser initialized.")
except Exception as e:
    print("Failed to init LSTMGuesser (model may be missing):", e)
    guesser = LSTMGuesser(None, training_words)


LSTMGuesser initialized.


In [42]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################
        # Normalize spaces and ensure word string is passed to guesser as expected
        # (LSTMGuesser expects the same API mask format like '_ p p _ e ' or '_pp_e')
        # We simply forward `word` directly.

        # If we have an LSTMGuesser instance, use it
        if hasattr(self, "guesser") and self.guesser is not None:
            try:
                # ensure guesser uses the latest dictionary (in case you update it)
                self.guesser.words = self.full_dictionary
                next_letter = self.guesser.guess(word)
                # Safety: ensure returned value is a single lowercase alpha char
                if isinstance(next_letter, str) and len(next_letter) == 1 and next_letter.isalpha():
                    return next_letter.lower()
            except Exception as e:
                # if anything goes wrong with ML guesser, fall back
                print("Warning: LSTMGuesser failed, falling back to frequency guess. Error:", e)

        # --- Fallback frequency-based guess (non-ML) ---
        # self.full_dictionary_common_letter_sorted is a list of (char, count)
        for tup in self.full_dictionary_common_letter_sorted:
            # each tup is (char, count)
            c = tup[0]
            if c not in self.guessed_letters:
                return c.lower()

        # Backup fallback: english frequency order
        default_rank = "etaoinshrdlcumwfgypbvkjxqz"
        for c in default_rank:
            if c not in self.guessed_letters:
                return c

#         # Final fallback: random remaining letter
#         import random
#         for c in "abcdefghijklmnopqrstuvwxyz":
#             if c not in self.guessed_letters:
#                 return c

        
    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
        
        # Reset LSTMGuesser internal state for the new game (if present)
        if hasattr(self, "guesser") and self.guesser is not None:
            try:
                self.guesser.reset_game()
            except Exception:
                # safe guard - ignore if guesser has no reset_game
                pass
            
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [37]:
api = HangmanAPI(access_token="Insert_Your_API_Key", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [43]:

N_RUNS = 20  # Number of practice games to play
wins = 0

for i in range(N_RUNS):
    api.start_game(practice=1,verbose=True)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / total_practice_runs
    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))




Successfully start a new game! Game ID: 5fde036db6f2. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': '5fde036db6f2', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ e _ '}
Guessing letter: i
Sever response: {'game_id': '5fde036db6f2', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ i _ e _ '}
Guessing letter: a
Sever response: {'game_id': '5fde036db6f2', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ i _ e _ '}
Guessing letter: n
Sever response: {'game_id': '5fde036db6f2', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ i n e _ '}
Guessing letter: o
Sever response: {'game_id': '5fde036db6f2', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ o _ _ _ i n e _ '}
Guessing letter: r
Sever response: {'game_id': '5fde036db6f2', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ o _ _ _ i n e _ '}
Guessing letter: s
Sever response: {'game_id': '5fde036db6f2', 'status': 'ongoing', 'tries_

Sever response: {'game_id': '7d81d14a155f', 'status': 'ongoing', 'tries_remains': 1, 'word': 't _ a t _ _ _ o r _ '}
Guessing letter: c
Sever response: {'game_id': '7d81d14a155f', 'status': 'ongoing', 'tries_remains': 1, 'word': 't _ a t c _ _ o r _ '}
Guessing letter: u
Sever response: {'game_id': '7d81d14a155f', 'status': 'failed', 'tries_remains': 0, 'word': 't _ a t c _ _ o r _ '}
Failed game: 7d81d14a155f. Because of: # of tries exceeded!
run 140 practice games out of an allotted 100,000. practice success rate so far = 0.171
Successfully start a new game! Game ID: 39d9ef86741b. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': '39d9ef86741b', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ _ '}
Guessing letter: i
Sever response: {'game_id': '39d9ef86741b', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ i _ i _ _ '}
Guessing letter: a
Sever response: {'game_id': '39d9ef86741b', 'status': 'ongoing', '

Sever response: {'game_id': '6ad3f635bba4', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ e n _ o n i t r i _ e '}
Guessing letter: l
Sever response: {'game_id': '6ad3f635bba4', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ e n _ o n i t r i l e '}
Guessing letter: c
Sever response: {'game_id': '6ad3f635bba4', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ e n _ o n i t r i l e '}
Guessing letter: u
Sever response: {'game_id': '6ad3f635bba4', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ e n _ o n i t r i l e '}
Guessing letter: d
Sever response: {'game_id': '6ad3f635bba4', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ e n _ o n i t r i l e '}
Guessing letter: p
Sever response: {'game_id': '6ad3f635bba4', 'status': 'failed', 'tries_remains': 0, 'word': '_ e n _ o n i t r i l e '}
Failed game: 6ad3f635bba4. Because of: # of tries exceeded!
run 145 practice games out of an allotted 100,000. practice success rate so far = 0.166
Successfully start a new game! G

Sever response: {'game_id': 'a7eedc8578fe', 'status': 'ongoing', 'tries_remains': 4, 'word': 'n i _ _ _ _ _ _ '}
Guessing letter: o
Sever response: {'game_id': 'a7eedc8578fe', 'status': 'ongoing', 'tries_remains': 4, 'word': 'n i _ _ o _ _ _ '}
Guessing letter: r
Sever response: {'game_id': 'a7eedc8578fe', 'status': 'ongoing', 'tries_remains': 4, 'word': 'n i _ r o _ _ _ '}
Guessing letter: s
Sever response: {'game_id': 'a7eedc8578fe', 'status': 'ongoing', 'tries_remains': 3, 'word': 'n i _ r o _ _ _ '}
Guessing letter: t
Sever response: {'game_id': 'a7eedc8578fe', 'status': 'ongoing', 'tries_remains': 3, 'word': 'n i t r o _ _ _ '}
Guessing letter: l
Sever response: {'game_id': 'a7eedc8578fe', 'status': 'ongoing', 'tries_remains': 3, 'word': 'n i t r o _ _ l '}
Guessing letter: c
Sever response: {'game_id': 'a7eedc8578fe', 'status': 'ongoing', 'tries_remains': 2, 'word': 'n i t r o _ _ l '}
Guessing letter: u
Sever response: {'game_id': 'a7eedc8578fe', 'status': 'ongoing', 'tries_rema

Sever response: {'game_id': '7f409449fca6', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ e _ i _ _ a '}
Guessing letter: n
Sever response: {'game_id': '7f409449fca6', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e _ i _ _ a '}
Guessing letter: o
Sever response: {'game_id': '7f409449fca6', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ e _ i _ _ a '}
Guessing letter: r
Sever response: {'game_id': '7f409449fca6', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ e r i _ _ a '}
Guessing letter: s
Sever response: {'game_id': '7f409449fca6', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ e r i _ _ a '}
Guessing letter: t
Sever response: {'game_id': '7f409449fca6', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ e r i _ _ a '}
Guessing letter: l
Sever response: {'game_id': '7f409449fca6', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ e r i l l a '}
Guessing letter: c
Sever response: {'game_id': '7f409449fca6', 'status': 'success', 'tries_remains': 2, 'word

In [30]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: 737bbeb10afe. # of tries remaining: 6. Word: _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': '737bbeb10afe', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ '}
Guessing letter: i
Sever response: {'game_id': '737bbeb10afe', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ '}
Guessing letter: a
Sever response: {'game_id': '737bbeb10afe', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ _ _ '}
Guessing letter: n
Sever response: {'game_id': '737bbeb10afe', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ _ _ _ _ '}
Guessing letter: o
Sever response: {'game_id': '737bbeb10afe', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ o _ o _ '}
Guessing letter: r
Sever response: {'game_id': '737bbeb10afe', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ o _ o _ '}
Guessing letter: s
Sever response: {'game_id': '737bbeb10afe', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ o _ o s '}
Guessing letter: t
Se

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [44]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Playing  1  th game
Playing  2  th game
Playing  3  th game
Playing  4  th game
Playing  5  th game
Playing  6  th game
Playing  7  th game
Playing  8  th game
Playing  9  th game
Playing  10  th game
Playing  11  th game
Playing  12  th game
Playing  13  th game
Playing  14  th game
Playing  15  th game
Playing  16  th game
Playing  17  th game
Playing  18  th game
Playing  19  th game
Playing  20  th game
Playing  21  th game
Playing  22  th game
Playing  23  th game
Playing  24  th game
Playing  25  th game
Playing  26  th game
Playing  27  th game
Playing  28  th game
Playing  29  th game
Playing  30  th game
Playing  31  th game
Playing  32  th game
Playing  33  th game
Playing  34  th game
Playing  35  th game
Playing  36  th game
Playing  37  th game
Playing  38  th game
Playing  39  th game
Playing  40  th game
Playing  41  th game
Playing  42  th game
Playing  43  th game
Playing  44  th game
Playing  45  th game
Playing  46  th game
Playing  47  th game
Pl

Playing  378  th game
Playing  379  th game
Playing  380  th game
Playing  381  th game
Playing  382  th game
Playing  383  th game
Playing  384  th game
Playing  385  th game
Playing  386  th game
Playing  387  th game
Playing  388  th game
Playing  389  th game
Playing  390  th game
Playing  391  th game
Playing  392  th game
Playing  393  th game
Playing  394  th game
Playing  395  th game
Playing  396  th game
Playing  397  th game
Playing  398  th game
Playing  399  th game
Playing  400  th game
Playing  401  th game
Playing  402  th game
Playing  403  th game
Playing  404  th game
Playing  405  th game
Playing  406  th game
Playing  407  th game
Playing  408  th game
Playing  409  th game
Playing  410  th game
Playing  411  th game
Playing  412  th game
Playing  413  th game
Playing  414  th game
Playing  415  th game
Playing  416  th game
Playing  417  th game
Playing  418  th game
Playing  419  th game
Playing  420  th game
Playing  421  th game
Playing  422  th game
Playing  4

Playing  751  th game
Playing  752  th game
Playing  753  th game
Playing  754  th game
Playing  755  th game
Playing  756  th game
Playing  757  th game
Playing  758  th game
Playing  759  th game
Playing  760  th game
Playing  761  th game
Playing  762  th game
Playing  763  th game
Playing  764  th game
Playing  765  th game
Playing  766  th game
Playing  767  th game
Playing  768  th game
Playing  769  th game
Playing  770  th game
Playing  771  th game
Playing  772  th game
Playing  773  th game
Playing  774  th game
Playing  775  th game
Playing  776  th game
Playing  777  th game
Playing  778  th game
Playing  779  th game
Playing  780  th game
Playing  781  th game
Playing  782  th game
Playing  783  th game
Playing  784  th game
Playing  785  th game
Playing  786  th game
Playing  787  th game
Playing  788  th game
Playing  789  th game
Playing  790  th game
Playing  791  th game
Playing  792  th game
Playing  793  th game
Playing  794  th game
Playing  795  th game
Playing  7

HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [8]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

ZeroDivisionError: division by zero